In [ ]:
import pandas as pd
import tensorflow as tf
import os

In [ ]:
#Hyperparameters
output_dir = "./"
test_dir = "../input/plant-pathology-2021-fgvc8/test_images/"
model_dir = "../input/conve01/eff7-e17/epoch-17"
efficientB7 = "../input/efficientb7/effb7"

image_dims = (300,300,3)
data_set = pd.read_csv("../input/plant-pathology-2021-fgvc8/train.csv")
df_labels = data_set['labels']
one_hot = df_labels.str.get_dummies(sep=" ")
dataset_labels = one_hot.columns.to_list()

In [ ]:
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, GlobalMaxPool2D, Conv2D, InputLayer
from tensorflow import concat


class MultiLabel(Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_backbone = tf.keras.models.load_model(efficientB7)
        self.model = Sequential()
        self.model.add(InputLayer(input_shape=image_dims))
        self.model.add(self.model_backbone)
        self.model.add(Conv2D(filters=1024, kernel_size=(1, 1), padding='same'))
        self.model.add(BatchNormalization(momentum=0.7))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(filters=1024, kernel_size=(1, 1), padding='same'))
        self.model.add(Conv2D(filters=2400, kernel_size=(1, 1), padding='same'))

        self.model_pred1_2 = Conv2D(filters=1024, kernel_size=(1, 1), padding='same')
        self.model_pred1_3 = BatchNormalization()
        self.model_pred1_4 = Conv2D(filters=512, kernel_size=(1, 1), padding='same')
        self.model_pred1_5 = GlobalMaxPool2D()
        self.model_pred1_6 = Dense(units=1, activation='sigmoid')

        self.model_pred2_2 = Conv2D(filters=1024, kernel_size=(1,1), padding='same')
        self.model_pred2_3 = BatchNormalization()
        self.model_pred2_4 = Conv2D(filters=512, kernel_size=(1,1), padding='same')
        self.model_pred2_5 = GlobalMaxPool2D()
        self.model_pred2_6 = Dense(units=1, activation='sigmoid')

        self.model_pred3_2 = Conv2D(filters=1024, kernel_size=(1,1), padding='same')
        self.model_pred3_3 = BatchNormalization()
        self.model_pred3_4 = Conv2D(filters=512, kernel_size=(1,1), padding='same')
        self.model_pred3_5 = GlobalMaxPool2D()
        self.model_pred3_6 = Dense(units=1, activation='sigmoid')

        self.model_pred4_2 = Conv2D(filters=1024, kernel_size=(1,1), padding='same')
        self.model_pred4_3 = BatchNormalization()
        self.model_pred4_4 = Conv2D(filters=512, kernel_size=(1,1), padding='same')
        self.model_pred4_5 = GlobalMaxPool2D()
        self.model_pred4_6 = Dense(units=1, activation='sigmoid')

        self.model_pred5_2 = Conv2D(filters=1024, kernel_size=(1,1), padding='same')
        self.model_pred5_3 = BatchNormalization()
        self.model_pred5_4 = Conv2D(filters=512, kernel_size=(1,1), padding='same')
        self.model_pred5_5 = GlobalMaxPool2D()
        self.model_pred5_6 = Dense(units=1, activation='sigmoid')

        self.model_pred6_2 = Conv2D(filters=1024, kernel_size=(1,1), padding='same')
        self.model_pred6_3 = BatchNormalization()
        self.model_pred6_4 = Conv2D(filters=512, kernel_size=(1,1), padding='same')
        self.model_pred6_5 = GlobalMaxPool2D()
        self.model_pred6_6 = Dense(units=1, activation='sigmoid')


        """
        self.model.add(self.model_backbone)
        self.model.add(Conv2D(filters=1024, kernel_size=(1, 1), padding='same'))
        self.model.add(BatchNormalization(momentum=0.7))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(filters=1024, kernel_size=(1, 1), padding='same'))
        self.model.add(Conv2D(filters=512, kernel_size=(1, 1), padding='same'))
        self.model.add(BatchNormalization(momentum=0.7))
        self.model.add(Dropout(0.2))
        self.model.add(Conv2D(filters=256, kernel_size=(1, 1), padding="same"))
        self.model.add(GlobalMaxPool2D())
        self.model.add(Dense(num_classes, activation='sigmoid'))
        self.model.summary()
        """

    def call(self, x, **kwargs):
        y = self.model(x)
        pred1 = y[:,:,:,:400]
        pred2 = y[:,:,:,400:800]
        pred3 = y[:,:,:,800:1200]
        pred4 = y[:,:,:,1200:1600]
        pred5 = y[:,:,:,1600:2000]
        pred6 = y[:,:,:,2000:2400]

        pred1 = self.model_pred1_2(pred1)
        pred1 = self.model_pred1_3(pred1)
        pred1 = self.model_pred1_4(pred1)
        pred1 = self.model_pred1_5(pred1)
        pred1 = self.model_pred1_6(pred1)

        pred2 = self.model_pred2_2(pred2)
        pred2 = self.model_pred2_3(pred2)
        pred2 = self.model_pred2_4(pred2)
        pred2 = self.model_pred2_5(pred2)
        pred2 = self.model_pred2_6(pred2)

        pred3 = self.model_pred3_2(pred3)
        pred3 = self.model_pred3_3(pred3)
        pred3 = self.model_pred3_4(pred3)
        pred3 = self.model_pred3_5(pred3)
        pred3 = self.model_pred3_6(pred3)

        pred4 = self.model_pred4_2(pred4)
        pred4 = self.model_pred4_3(pred4)
        pred4 = self.model_pred4_4(pred4)
        pred4 = self.model_pred4_5(pred4)
        pred4 = self.model_pred4_6(pred4)

        pred5 = self.model_pred5_2(pred5)
        pred5 = self.model_pred5_3(pred5)
        pred5 = self.model_pred5_4(pred5)
        pred5 = self.model_pred5_5(pred5)
        pred5 = self.model_pred5_6(pred5)

        pred6 = self.model_pred6_2(pred6)
        pred6 = self.model_pred6_3(pred6)
        pred6 = self.model_pred6_4(pred6)
        pred6 = self.model_pred6_5(pred6)
        pred6 = self.model_pred6_6(pred6)

        return concat([pred1, pred2, pred3, pred4,pred5, pred6],axis=1)

    def create_model(self):
        return self.model


In [ ]:
if __name__ == '__main__':

    # Initialization of model and data creator class.
    model = MultiLabel()
    model.build(input_shape=[None, image_dims[0], image_dims[1], image_dims[2]])
    model.load_weights(model_dir)
    images_path_list = sorted(list(os.listdir(test_dir)))
    def test_on_sub (index):
        input_img = tf.io.read_file(test_dir+images_path_list[index])
        image = tf.io.decode_image(contents=input_img, channels=3, dtype=tf.dtypes.float32)
        tensor_image = tf.image.resize(image, [image_dims[0], image_dims[1]])
        name_jpg = images_path_list[index].split(os.path.sep)[-1]
        return name_jpg, tf.expand_dims(tensor_image, axis=0)
    
    values = []
    for i in range(len(images_path_list)):
        name, images = test_on_sub(index=i)
        images = images * 255
        test_values = model.call(images)
        print(test_values)
        index_values = [i for i, v in enumerate(test_values[0]) if v > 0.7]
        classes = dataset_labels
        classes_img = ""
        for i in index_values:
            classes_img = str(classes[i])+" "+classes_img
        values.append([name, classes_img])
    csv_pd = pd.DataFrame(values, columns=['image', 'labels'], index=None)
    csv_pd.to_csv(output_dir + 'submission.csv',index=False)